In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def calculate_rsi(df, period=14):
    delta = df['Close'].diff()
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    avg_gain = gains.rolling(window=period).mean()
    avg_loss = losses.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

def plot_rsi(df, ticker):
    plt.figure(figsize=(14, 7))
    plt.subplot(2, 1, 1)
    plt.plot(df['Close'], label=f'{ticker} Closing Price', color='blue')
    plt.title(f'{ticker} Price and RSI')
    plt.legend()
    plt.grid()

    plt.subplot(2, 1, 2)
    plt.plot(df['RSI'], label='RSI', color='orange')
    plt.axhline(70, linestyle='--', color='red', label='Overbought (70)')
    plt.axhline(30, linestyle='--', color='green', label='Oversold (30)')
    plt.title('Relative Strength Index (RSI)')
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()

def fetch_and_plot_rsi(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data = data[['Close']]
    df = calculate_rsi(data)
    plot_rsi(df, ticker)

ticker_input = widgets.Text(description='Ticker:', continuous_update=False)
start_date_input = widgets.Text(description='Start Date:', continuous_update=False)
end_date_input = widgets.Text(description='End Date:', continuous_update=False)
compute_button = widgets.Button(description='Compute RSI', button_style='primary')

output = widgets.Output()

def on_compute_clicked(b):
    with output:
        output.clear_output()
        fetch_and_plot_rsi(ticker_input.value, start_date_input.value, end_date_input.value)

compute_button.on_click(on_compute_clicked)

input_box = widgets.VBox([ticker_input, start_date_input, end_date_input, compute_button])
display(widgets.VBox([
    widgets.HBox([input_box]),
    widgets.HBox([output])
]))